In [1]:
# Dependencies
import sqlalchemy
from flask import Flask, jsonify, render_template
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd

In [2]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Create our session (link) from Python to the DB
session = Session(engine)

In [3]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# Get the table names from the database
table_names = Base.classes.keys()

/var/folders/ds/jk39g9xd62v_5kxdlxcxn0800000gn/T/ipykernel_26001/1483291640.py:3: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [4]:
# Save references to each table
M_base = Base.classes.measurement
S_base = Base.classes.station

In [5]:
# Import Measurement data
measure_col_names = [column.key for column in M_base.__table__.columns]
measure_rows = session.query(M_base).all()
measure_data = [m.__dict__ for m in measure_rows]
Measurement = pd.DataFrame(measure_data, columns=measure_col_names)

# Import Station data
station_col_names = [column.key for column in S_base.__table__.columns]
station_rows = session.query(S_base).all()
station_data = [s.__dict__ for s in station_rows]
Station = pd.DataFrame(station_data, columns=station_col_names)

In [6]:
# create Flask app

app = Flask(__name__)

In [7]:

@app.route("/")
def home():
    routes = [rule.rule for rule in app.url_map.iter_rules() if "static" not in rule.rule]
    routes_html = "\n".join([f"<p style='font-size: 30px;'>{url}</p>" for url in routes])
    return render_template("home.html", routes_html=routes_html)


@app.route("/second")
def second():
    print("Server received request for SECOND page...")
    return "Welcome to the SECOND page!"

@app.route("/measurement")
def display_measurement():
    mcol = Measurement.columns.tolist()  # Get column names
    mrows = Measurement.to_dict(orient='records')  # Convert DataFrame to list of dictionaries
    return render_template("measurement.html", columns=mcol, rows=mrows)

@app.route("/station")
def display_station():
    scol = Station.columns.tolist()  # Get column names
    srows = Station.to_dict(orient='records')  # Convert DataFrame to list of dictionaries
    return render_template("station.html", columns=scol, rows=srows)


In [8]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Aug/2023 23:00:19] "GET /station HTTP/1.1" 200 -
127.0.0.1 - - [24/Aug/2023 23:00:24] "GET /second HTTP/1.1" 200 -


Server received request for SECOND page...


127.0.0.1 - - [24/Aug/2023 23:00:24] "GET / HTTP/1.1" 200 -


In [ ]:
session.close